In [1]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import JulierSigmaPoints, unscented_transform

Параметры закона распределения в исходной системе координат

In [2]:
mymean = [20.6, np.pi/2, np.pi/3]
mycov = [[1.55,0,0],
         [0,np.pi/10,0],
         [0,0,np.pi/15]]

Выборка для тестирования алгоритма

In [3]:
r, theta, phi = np.random.multivariate_normal(mean = mymean, cov= mycov, size=10000).T
r, theta, phi

(array([19.55573122, 18.05492431, 19.52706952, ..., 21.38151703,
        19.35422566, 19.8457244 ]),
 array([1.55346126, 1.93315373, 1.57636603, ..., 1.75425509, 2.46501742,
        1.55041848]),
 array([0.79357266, 1.47489445, 1.89493684, ..., 1.10050338, 0.74634614,
        1.0748861 ]))

Функция преобразования координат (может быть любой)

In [4]:
def f(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z

In [5]:
sigmas = JulierSigmaPoints(n=3, kappa=1.0) 
sigmas

JulierSigmaPoints object
n = 3
kappa = 1.0
Wm = [0.25  0.125 0.125 0.125 0.125 0.125 0.125]
Wc = [0.25  0.125 0.125 0.125 0.125 0.125 0.125]
subtract = <ufunc 'subtract'>
sqrt = <function cholesky at 0x7f9de982b280>

In [6]:
points = sigmas.sigma_points(np.array([np.mean(r), np.mean(theta), np.mean(phi)]), np.cov((r, theta, phi)))
points

array([[20.58042945,  1.56604676,  1.05234593],
       [23.09599141,  1.56612404,  1.05402675],
       [20.58042945,  2.68878075,  1.0605641 ],
       [20.58042945,  1.56604676,  1.96521147],
       [18.06486748,  1.56596949,  1.05066512],
       [20.58042945,  0.44331278,  1.04412777],
       [20.58042945,  1.56604676,  0.13948039]])

In [7]:
mp = np.stack(f(points[:,0],points[:,1],points[:,2]),axis = 1)
mp

array([[ 10.19820323,  17.8757146 ,   0.09774767],
       [ 11.41100733,  20.07989259,   0.10791076],
       [  4.39729998,   7.85704056, -18.50634331],
       [ -7.90831751,  19.00007989,   0.09774767],
       [  8.97802613,  15.675678  ,   0.08719581],
       [  4.43727426,   7.6313866 ,  18.59103579],
       [ 20.38033006,   2.86123545,   0.09774767]])

In [8]:
ukf_mean, ukf_cov = unscented_transform(mp, sigmas.Wm, sigmas.Wc)
ukf_mean, ukf_cov

(array([ 7.76150334, 13.60709279,  0.08384872]),
 array([[ 56.72753202, -16.74457521,   0.1420341 ],
        [-16.74457521,  36.99354639,  -0.43650065],
        [  0.1420341 ,  -0.43650065,  86.01407199]]))

Проверка

In [9]:
x, y, z = f(r, theta, phi)
np.mean(x), np.mean(y), np.mean(z)

(7.833731797464162, 13.774601171126758, 0.09637994130895981)

In [10]:
np.cov((x, y, z))

array([[ 45.79052798, -15.54003107,   0.33259348],
       [-15.54003107,  28.54891815,   0.20478675],
       [  0.33259348,   0.20478675,  99.69760814]])